In [176]:
from pyomo.environ import *
import pandas as pd
import matplotlib.pyplot as plt

In [177]:
b = 100
U_h = 3
U_up = 8
U_down = 6
R_up = 1.5
R_down = 1.5
u0 = 0

prob = [0.5, 0.5]

data = pd.read_excel('data.xlsx', index_col=0)
print(data)
data_prob = {'prob': [0.5, 0.5],
             't1': [120, 120],
             't2': [105, 45],
             't3': [154, 66],
             't4': [84, 36]
            }
data_prob = pd.DataFrame(data_prob, index=['s1','s2'])
#prices at time t
print(data_prob)

    price
t1    120
t2     75
t3    110
t4     60
    prob   t1   t2   t3  t4
s1   0.5  120  105  154  84
s2   0.5  120   45   66  36


In [178]:
model = ConcreteModel()

# DEFINE SETS
model.T_det = Set(ordered=True, initialize=data.index[:1])
model.T_stoch = Set(ordered=True, initialize=data.index[1:])
model.S = Set(ordered=True, initialize=data_prob.index)

# DEFINE PARAMETERS
model.lamb_stoch = Param(model.T_stoch, model.S, within=NonNegativeReals, mutable=True)
model.prob = Param(model.S, within=NonNegativeReals, mutable=True)

# INITIALIZE PARAMETERS
for t in model.T_stoch:
    for s in model.S:
        model.lamb_stoch[t,s] = data_prob.loc[s, t]

for s in model.S:
    model.prob[s] = data_prob.loc[s, 'prob']

# DEFINE VARIABLES
model.u_det = Var(model.T_det, within=NonNegativeReals)
model.u_stoch = Var(model.T_stoch, model.S, within=NonNegativeReals)

# DEFINE COST AND CONSTRAINTS
def cost_rule(model):
    return (120 - b)*model.u_det['t1'] + \
            sum(model.prob[s]*((model.lamb_stoch[t,s] - b)*model.u_stoch[t,s]) for t in model.T_stoch for s in model.S)
model.cost = Objective(rule=cost_rule)

def ramp_up_rule_det(model, t):
    return model.u_det[t] - u0 <= R_up
model.ramp_up_det = Constraint(model.T_det, rule=ramp_up_rule_det)

def ramp_up_rule_stoch(model, tdet, tstoch, s):
    if model.T_stoch.ord(tstoch) == 1:
        return model.u_stoch[tstoch, s] - model.u_det[tdet] <= R_up
    else:
        return model.u_stoch[tstoch, s] - model.u_stoch[model.T_stoch.prev(tstoch), s] <= R_up
model.ramp_up_stoch = Constraint(model.T_det, model.T_stoch, model.S, rule=ramp_up_rule_stoch)

def ramp_down_rule_det(model, t):
    return model.u_det[t] - u0 >= -R_down
model.ramp_down_det = Constraint(model.T_det, rule=ramp_down_rule_det)

def ramp_down_rule_stoch(model, tdet, tstoch, s):
    if model.T_stoch.ord(tstoch) == 1:
        return model.u_stoch[tstoch, s] - model.u_det[tdet] >= -R_down
    else:
        return model.u_stoch[tstoch, s] - model.u_stoch[model.T_stoch.prev(tstoch), s] >= -R_down
model.ramp_down_stoch = Constraint(model.T_det, model.T_stoch, model.S, rule=ramp_down_rule_stoch)

def minmax_cons_rule_det(model, t):
    return inequality(0, model.u_det[t], U_h)
model.minmax_const_det = Constraint(model.T_det, rule=minmax_cons_rule_det)

def minmax_cons_rule_stoch(model, t, s):
    return inequality(0, model.u_stoch[t,s], U_h)
model.minmax_const_stoch = Constraint(model.T_stoch, model.S, rule=minmax_cons_rule_stoch)

def minmax_total_rule(model, tdet, s):
    return inequality(U_down, model.u_det[tdet] + sum(model.u_stoch[t, s] for t in model.T_stoch), U_up)
model.minmax_total = Constraint(model.T_det, model.S, rule=minmax_total_rule)

In [179]:
model.pprint()

11 Set Declarations
    S : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=Insertion, Bounds=None
        ['s1', 's2']
    T_det : Dim=0, Dimen=1, Size=1, Domain=None, Ordered=Insertion, Bounds=None
        ['t1']
    T_stoch : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=Insertion, Bounds=None
        ['t2', 't3', 't4']
    lamb_stoch_index : Dim=0, Dimen=2, Size=6, Domain=None, Ordered=Insertion, Bounds=None
        Virtual
    minmax_const_stoch_index : Dim=0, Dimen=2, Size=6, Domain=None, Ordered=Insertion, Bounds=None
        Virtual
    minmax_total_index : Dim=0, Dimen=2, Size=2, Domain=None, Ordered=Insertion, Bounds=None
        Virtual
    ramp_down_stoch_index : Dim=0, Dimen=3, Size=6, Domain=None, Ordered=Insertion, Bounds=None
        Virtual
    ramp_down_stoch_index_index_0 : Dim=0, Dimen=2, Size=3, Domain=None, Ordered=Insertion, Bounds=None
        Virtual
    ramp_up_stoch_index : Dim=0, Dimen=3, Size=6, Domain=None, Ordered=Insertion, Bounds=None
        Virtual
    ra

In [180]:
opt = SolverFactory('gurobi')
results = opt.solve(model)

In [181]:
resData = pd.DataFrame(index=data_prob.index, columns=['t1', 't2', 't3', 't4', 'total'])

for s in model.S:
    for t in model.T_stoch:
        resData.loc[s, t] = value(model.u_stoch[t,s])
    resData.loc[s,'t1'] = value(model.u_det['t1'])
    resData.loc[s,'total'] = resData.loc[s,:].sum()
resData.columns = ['u1','u2','u3','u4','total']
print('Total consumption for each scenario at time period 1-4 and the final total consumption (constrained between 6, 8)')
resData

Total consumption for each scenario at time period 1-4 and the final total consumption (constrained between 6, 8)


,u1,u2,u3,u4,total
s1,0.25,1.75,1.25,2.75,6
s2,0.25,1.75,3,3,8
